<a href="https://colab.research.google.com/github/soyebganja/DL-Practice-Projects/blob/main/10%3AModel%20Optimization%3A%20Hyperparameter%20Tuning/10_3_Optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

!pip install optuna # Install the missing optuna library
import optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 16.0 MB/s eta 0:00:00


In [3]:
# 1. Generate sythetic dataset
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
X_train.shape

(800, 20)

In [5]:
X_train[0]

array([ 0.50363664, -1.51368248, -0.46907062,  1.90176571, -0.87064279,
        1.82004715,  1.66291365,  1.29105223, -0.16713608, -1.04718436,
        1.43003039,  0.20104766,  1.27577182, -1.13260729,  1.75008532,
       -1.4089039 ,  0.03301588, -0.80340946, -1.31410638,  1.41209637])

In [6]:
y_train[0]

np.int64(1)

In [7]:
# Convert to PyTorch tensors
X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long)
X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long)

In [8]:
class SimpleNN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 2)
        )

    def forward(self, x):
        return self.network(x)

In [9]:
def objective(trial):
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
  hidden_dim = trial.suggest_int('hidden_dim', 10, 100)

  model = SimpleNN(input_dim=20, hidden_dim=hidden_dim)
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)
  train_dataset = TensorDataset(X_train, y_train)


  # Trainingloop
  epochs = 20
  batch_size = 32
  train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
  test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=batch_size, shuffle=False)

  for epoch in range(epochs):
    model.train()
    for batch_x, batch_y in train_loader:
      optimizer.zero_grad()
      output = model(batch_x)
      loss = criterion(output, batch_y)
      loss.backward()
      optimizer.step()

  # validation accuracy
  model.eval()
  with torch.no_grad():
    correct = 0
    total = 0
    for batch_x, batch_y in test_loader:
      output = model(batch_x)
      _, predicted = torch.max(output.data, 1)
      total += batch_y.size(0)
      correct += (predicted == batch_y).sum().item()

  accuracy = correct / total

  return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)


[I 2025-04-20 15:28:59,073] A new study created in memory with name: no-name-e576d436-63dd-4b59-80fb-8129a63e46cd
<ipython-input-9-fd2fd2d7bf9c>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
[I 2025-04-20 15:29:00,027] Trial 0 finished with value: 0.84 and parameters: {'learning_rate': 0.002039425471673405, 'hidden_dim': 82}. Best is trial 0 with value: 0.84.
[I 2025-04-20 15:29:00,905] Trial 1 finished with value: 0.845 and parameters: {'learning_rate': 0.0018861769358919798, 'hidden_dim': 74}. Best is trial 1 with value: 0.845.
[I 2025-04-20 15:29:01,651] Trial 2 finished with value: 0.86 and parameters: {'learning_rate': 0.0008719762440083374, 'hidden_dim': 87}. Best is trial 2 with value: 0.86.
[I 2025-04-20 15:29:02,916] Trial 3 finished with value: 0

In [10]:
study.best_params

{'learning_rate': 0.0010814105555317093, 'hidden_dim': 67}